## Exploring two word2vec models (word2vec vs gensim)

In [1]:
import word2vec
import numpy

import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk

### 1. Python interface to Google word2vec

Ref: https://github.com/danielfrg/word2vec

In [2]:
def read_dataset(path):
    """Read a dataset, where the first column contains a real-valued score,
    followed by a tab and a string of words.
    """
    dataset = []
    with open(path, "r") as f:
        for line in f:
            line_parts = line.strip().split("\t")
            dataset.append((float(line_parts[0]), line_parts[1].lower()))
    return dataset

In [35]:
path = '../data/info_word2vec_vs_gensim/train.txt'
sentences_train = read_dataset(path)
models_path = '../trained_models/info/'

In [36]:
f = open(models_path + 'sentence_forword2vector.txt', 'w')
for label, sentence in sentences_train:
    f.write(sentence)
f.close()

word2phrase groups up similar words "Los Angeles" to "Los_Angeles".

**Note:** word2phrase will create a phrases text file which can be used as a better input for word2vec. However, we can also use the original text file as input for word2vec, thus this step can be skipped.

In [37]:
word2vec.word2phrase(models_path + 'sentence_forword2vector.txt', models_path + 'sentence_phrases.txt', verbose=True)

Starting training using file ../trained_models/info/sentence_forword2vector.txt
Words processed: 100K     Vocab size: 76K  
Vocab size (unigrams + bigrams): 51508
Words in train file: 133711


Train the model using the word2phrase output.

word2vec generates a bin file containing the word vectors in a binary format.

In [38]:
word2vec.word2vec(models_path + 'sentence_phrases.txt', models_path + 'sentence_phrases_bin.bin', size=100, verbose=True)

Starting training using file ../trained_models/info/sentence_phrases.txt
Vocab size: 3062
Words in train file: 110763


word2clusters cluster the trained vectors.

The output file contains the cluster for every word in the vocabulary.

In [39]:
word2vec.word2clusters(models_path + 'sentence_phrases_bin.bin', models_path + 'sentence_phrases_clusters.txt', 100, verbose=True)

Starting training using file ../trained_models/info/sentence_phrases_bin.bin
Vocab size: 10
Words in train file: 7315


In [40]:
model = word2vec.load(models_path + 'sentence_phrases_bin.bin')
model.vocab

/Users/lifa08/anaconda3/lib/python3.6/site-packages/word2vec/utils.py:7: RuntimeWarning: divide by zero encountered in true_divide
  return (1.0 / LA.norm(vec, ord=2)) * vec
/Users/lifa08/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2168: RuntimeWarning: invalid value encountered in sqrt
  ret = sqrt(sqnorm)


array(['</s>', ',', 'the', ..., 'ethnic', 'nonsense', 'earth'],
      dtype='<U78')

In [41]:
model.vectors.shape

(3062, 100)

In [42]:
model.vectors

array([[  6.58986646e-13,   7.27593387e-13,  -6.30599360e-13, ...,
          2.50888205e-13,   5.03685635e-13,   3.18791556e-14],
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf],
       [ -5.62611520e-11,  -5.00759087e-11,   3.81477835e-11, ...,
         -2.24868180e-11,  -4.26854073e-11,  -6.83195792e-11],
       ..., 
       [             inf,             -inf,              inf, ...,
                     inf,             -inf,             -inf],
       [  4.68127880e+07,  -2.65415392e+08,   9.31840160e+07, ...,
          1.49419648e+08,  -1.03577656e+08,  -2.93692864e+08],
       [  2.17401704e-12,  -1.59137998e-11,   5.15266085e-12, ...,
          9.41512753e-12,  -7.24156499e-12,  -1.71239758e-11]])

Retreive the vector of individual words.

In [43]:
model['nonsense'].shape

(100,)

In [44]:
model[','][:10]

array([ inf, -inf,  inf, -inf,  inf,  inf, -inf, -inf, -inf,  inf])

Do simple queries to retreive words similar to "the" based on cosine similarity.

This returned a tuple with 2 items:

    indexes: numpy array with the indexes of the similar words in the vocabulary
    metrics: numpy array with cosine similarity to each word

In [45]:
indexes, metrics = model.cosine(',')
indexes, metrics

(array([1016, 1025, 1024, 1023, 1022, 1021, 1020, 1019, 1018, 1017]),
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]))

In [46]:
model.vocab[indexes]

array(['faith', 'writer', 'class', 'ultimate', 'cute', 'fears', 'fit',
       '.nothing', 'steven', 'several'],
      dtype='<U78')

In [47]:
model.generate_response(indexes, metrics)

rec.array([('faith',  nan), ('writer',  nan), ('class',  nan),
           ('ultimate',  nan), ('cute',  nan), ('fears',  nan),
           ('fit',  nan), ('.nothing',  nan), ('steven',  nan),
           ('several',  nan)], 
          dtype=[('word', '<U78'), ('metric', '<f8')])

In [48]:
model.generate_response(indexes, metrics).tolist()

[('faith', nan),
 ('writer', nan),
 ('class', nan),
 ('ultimate', nan),
 ('cute', nan),
 ('fears', nan),
 ('fit', nan),
 ('.nothing', nan),
 ('steven', nan),
 ('several', nan)]

In [49]:
indexes, metrics = model.cosine('good')
model.generate_response(indexes, metrics).tolist()

[('faith', nan),
 ('writer', nan),
 ('class', nan),
 ('ultimate', nan),
 ('cute', nan),
 ('fears', nan),
 ('fit', nan),
 ('.nothing', nan),
 ('steven', nan),
 ('several', nan)]

### 2. gensim word2vec model

Refs:

https://radimrehurek.com/gensim/models/word2vec.html

https://rare-technologies.com/word2vec-tutorial/

In [50]:
sentences_train = read_dataset(path)

In [51]:
print(sentences_train[:2])

[(0.69444, "the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal ."), (0.83333, "the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth .")]


In [52]:
sentences = []
i = 0
for label, sentence in sentences_train:
    sentences.append(sentence)

print(sentences[:2])

["the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .", "the gorgeously elaborate continuation of `` the lord of the rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director peter jackson 's expanded vision of j.r.r. tolkien 's middle-earth ."]


In [53]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [54]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
print(tokenized_sentences[:2])

[['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], ['the', 'gorgeously', 'elaborate', 'continuation', 'of', '``', 'the', 'lord', 'of', 'the', 'rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co-writer\\/director', 'peter', 'jackson', "'s", 'expanded', 'vision', 'of', 'j.r.r', '.', 'tolkien', "'s", 'middle-earth', '.']]


**Note**: 

`gensim.models.Word2Vec(sentences, iter)` will run **two passes** over the sentences 
iterator (or, in general iter+1 passes; default iter=5). 

First pass: collects words and their frequencies to build **an internal dictionary tree**

The second and subsequent passes: train the neural model.

In [55]:
embedding_size = 10
model = gensim.models.Word2Vec(tokenized_sentences[0:5], min_count=1, size=embedding_size, window=5, workers=4)

2018-07-10 11:11:18,262 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-07-10 11:11:18,265 : INFO : collecting all words and their counts
2018-07-10 11:11:18,268 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-10 11:11:18,270 : INFO : collected 100 word types from a corpus of 140 raw words and 5 sentences
2018-07-10 11:11:18,273 : INFO : Loading a fresh vocabulary
2018-07-10 11:11:18,275 : INFO : min_count=1 retains 100 unique words (100% of original 100, drops 0)
2018-07-10 11:11:18,278 : INFO : min_count=1 leaves 140 word corpus (100% of original 140, drops 0)
2018-07-10 11:11:18,283 : INFO : deleting the raw counts dictionary of 100 items
2018-07-10 11:11:18,288 : INFO : sample=0.001 downsamples 100 most-common words
2018-07-10 11:11:18,292 : INFO : downsampling leaves estimated 55 word corpus (40.0% of prior 140)
2018-07-10 11:11:18,296 : INFO : estimated required memory for 100 words and 10 dimensions: 5800

Access the word vector for an individual word.

In [56]:
model.wv['but']

array([ 0.0402225 , -0.01383673, -0.04880496,  0.03738531, -0.0256329 ,
       -0.02597997, -0.04470853,  0.00275434, -0.04621249, -0.01988631], dtype=float32)

In [57]:
model.save(models_path + 'mygword2vmodel')

2018-07-10 11:11:18,583 : INFO : saving Word2Vec object under ../trained_models/info/mygword2vmodel, separately None
2018-07-10 11:11:18,589 : INFO : not storing attribute syn0norm
2018-07-10 11:11:18,592 : INFO : not storing attribute cum_table
2018-07-10 11:11:18,598 : INFO : saved ../trained_models/info/mygword2vmodel


In [58]:
del model

In [59]:
model = gensim.models.Word2Vec.load(models_path + 'mygword2vmodel')

2018-07-10 11:11:27,533 : INFO : loading Word2Vec object from ../trained_models/info/mygword2vmodel
2018-07-10 11:11:27,538 : INFO : loading wv recursively from ../trained_models/info/mygword2vmodel.wv.* with mmap=None
2018-07-10 11:11:27,541 : INFO : setting ignored attribute syn0norm to None
2018-07-10 11:11:27,543 : INFO : setting ignored attribute cum_table to None
2018-07-10 11:11:27,547 : INFO : loaded ../trained_models/info/mygword2vmodel


In [60]:
model.wv['but']

array([ 0.0402225 , -0.01383673, -0.04880496,  0.03738531, -0.0256329 ,
       -0.02597997, -0.04470853,  0.00275434, -0.04621249, -0.01988631], dtype=float32)

`model.wv` is a dictionary that contains `model.wv.index2word` and `model.wv.syn0`.

`model.wv.syn0` contains the word embeddings and is thus of shape (num_words, embedding_size)

In [61]:
# create a dictionary that maps words to their corresponding embedding vectors
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
print(w2v['but'])
print(model.wv.syn0.shape)
print(model.wv.syn0[:3])

[ 0.0402225  -0.01383673 -0.04880496  0.03738531 -0.0256329  -0.02597997
 -0.04470853  0.00275434 -0.04621249 -0.01988631]
(100, 10)
[[ -4.85028736e-02   6.90227933e-03   3.76802385e-02  -1.43604996e-02
    1.30194193e-02  -1.02926986e-02   4.54240888e-02  -2.78824475e-02
    7.39598254e-05   4.64480370e-02]
 [  2.34042350e-02   3.70108224e-02   3.80503125e-02   2.76730284e-02
   -3.75781469e-02  -2.50255018e-02  -1.28103625e-02   2.12863479e-02
   -2.23294012e-02  -3.30078602e-02]
 [ -4.36908379e-02   2.53004041e-02   2.75953971e-02  -2.04372723e-02
   -1.22186681e-02   2.49917340e-02   4.67930268e-03   1.70949602e-03
    1.95773505e-02  -5.59843145e-03]]


In [62]:
# http://adventuresinmachinelearning.com/gensim-word2vec-tutorial/
embedding_matrix = numpy.zeros((len(model.wv.vocab), embedding_size))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix[:3])

print(numpy.allclose(embedding_matrix, model.wv.syn0))

[[ -4.85028736e-02   6.90227933e-03   3.76802385e-02  -1.43604996e-02
    1.30194193e-02  -1.02926986e-02   4.54240888e-02  -2.78824475e-02
    7.39598254e-05   4.64480370e-02]
 [  2.34042350e-02   3.70108224e-02   3.80503125e-02   2.76730284e-02
   -3.75781469e-02  -2.50255018e-02  -1.28103625e-02   2.12863479e-02
   -2.23294012e-02  -3.30078602e-02]
 [ -4.36908379e-02   2.53004041e-02   2.75953971e-02  -2.04372723e-02
   -1.22186681e-02   2.49917340e-02   4.67930268e-03   1.70949602e-03
    1.95773505e-02  -5.59843145e-03]]
True


Transfer sentence words to their corresponding embeddings.

In [63]:
sentence_matrix = numpy.zeros((len(tokenized_sentences[0]), embedding_size))
for i, word in enumerate(tokenized_sentences[0]):
    sentence_matrix[i] = model.wv[word]
    # sentence_matrix[i] = w2v[word]
print(sentence_matrix[:2])

sentence_wv = model.wv[tokenized_sentences[0]]
print(sentence_wv[:2])

print(numpy.allclose(sentence_matrix, sentence_wv))

[[ -4.85028736e-02   6.90227933e-03   3.76802385e-02  -1.43604996e-02
    1.30194193e-02  -1.02926986e-02   4.54240888e-02  -2.78824475e-02
    7.39598254e-05   4.64480370e-02]
 [ -3.41202095e-02  -3.38358618e-02  -1.68177821e-02  -3.70543115e-02
   -4.28975783e-02  -2.08996329e-02   2.73697879e-02  -4.39694040e-02
   -3.52290869e-02  -2.83895321e-02]]
[[ -4.85028736e-02   6.90227933e-03   3.76802385e-02  -1.43604996e-02
    1.30194193e-02  -1.02926986e-02   4.54240888e-02  -2.78824475e-02
    7.39598254e-05   4.64480370e-02]
 [ -3.41202095e-02  -3.38358618e-02  -1.68177821e-02  -3.70543115e-02
   -4.28975783e-02  -2.08996329e-02   2.73697879e-02  -4.39694040e-02
   -3.52290869e-02  -2.83895321e-02]]
True


In [64]:
print(len(model.wv.vocab))

idx_sentence = numpy.zeros(len(tokenized_sentences[0]))
for i, word in enumerate(tokenized_sentences[0]):
    idx_sentence[i] = model.wv.vocab[word].index
print(idx_sentence)

100
[  0.  13.   6.  14.   7.  15.   0.  16.  17.   4.  18.   8.  19.   9.  20.
  10.  21.   4.  22.   7.  23.   3.  24.  25.  26.  27.  28.  29.   5.  30.
  31.  32.  33.  34.  35.   2.]


In [65]:
embedding_matrix[35]

array([-0.0079626 , -0.01740446,  0.03870961, -0.04500164, -0.03749513,
        0.03458533,  0.04255144,  0.02880084,  0.00620809,  0.01464351])

In [66]:
print(len(tokenized_sentences))

7161


In [67]:
def sentences_to_idxs(tokenized_sentences):
    idx_sentences = []
    for tokenized_sentence in tokenized_sentences:
        idx_one_sentence = numpy.zeros(len(tokenized_sentence))
        idx = 0
        for idx, word in enumerate(tokenized_sentence):
            idx_one_sentence[idx] = model.wv.vocab[word].index
        idx_sentences.append(idx_one_sentence) 
    return idx_sentences

In [68]:
idx_sentences = sentences_to_idxs(tokenized_sentences[0:5])
print(idx_sentences)

[array([  0.,  13.,   6.,  14.,   7.,  15.,   0.,  16.,  17.,   4.,  18.,
         8.,  19.,   9.,  20.,  10.,  21.,   4.,  22.,   7.,  23.,   3.,
        24.,  25.,  26.,  27.,  28.,  29.,   5.,  30.,  31.,  32.,  33.,
        34.,  35.,   2.]), array([  0.,  36.,  37.,  38.,   1.,   8.,   0.,  39.,   1.,   0.,  40.,
         9.,  41.,   6.,  42.,  43.,  10.,   3.,  44.,   1.,  45.,  46.,
        47.,  48.,  49.,  50.,  51.,  52.,   4.,  53.,  54.,   1.,  55.,
         2.,  56.,   4.,  57.,   2.]), array([ 58.,  59.,  60.,  61.,   3.,  62.,   1.,  63.,  11.,   3.,  12.,
        64.,  65.,   5.,   3.,  12.,  66.,  67.,  68.,   7.,   0.,  69.,
        11.,  70.,   0.,  71.,  72.,  73.,  74.,   0.,  75.,   5.,  76.,
         5.,  77.,   1.,   0.,  78.,   2.]), array([ 79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,   1.,
        89.,  90.,  91.,  92.,  93.,   1.,  94.,   2.]), array([ 95.,   0.,  96.,   6.,  97.,  98.,  99.,   2.])]


word2vec model does not need to tokenize sentence before training while gensim model expects a sequence of sentences which are composed of a list of words. Therefore, gensim model is often used together with natural language processing tools such as nltk. From the above explorations, we can see that gensim model is more stable.